In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from flaml import AutoML
from catboost import CatBoostRegressor
import os

In [15]:
file_path = f"{'/'.join(os.getcwd().split('/')[:3])}/year_project"
df = pd.read_csv(f'{file_path}/csv_data/air_weather_data_lite.csv')

### 1. Подготовка данных

#### 1.1 Заполняем пропуски

Для каждого столбца разный алгоритм обработки пропусков<br>
* Строки с пропусками в столбце european_aqi можем удалить
* Строки с глубиной снега заполняем 0, так как пропуски в июле и августе
* Строки со скоростью ветра заполняем медианным значением региона по году и месяцу
* Пропуски в поле income заполняем медианным значением по году по региону
* Пропуски в поле nitrogen_monoxide заполняем медианным значением по году и региону

In [16]:
df.dropna(subset=['european_aqi'], inplace=True)
df['snow_depth'] = df['snow_depth'].fillna(0)

wind_median = df.groupby(['region', 'year', 'month'])['snow_depth'].transform('median')
df['wind_speed_10m'] = df['snow_depth'].fillna(wind_median)

median_income_by_region_year = df.groupby(['region', 'year'])['income'].median().reset_index().rename(columns={'income': 'median_income'})
df = df.merge(median_income_by_region_year, on=['region', 'year'], how='left')
df['income'] = df['income'].fillna(df['median_income'])
df = df.drop(columns=['median_income'])

median_monoxide_by_region_year = df.groupby(['region', 'year'])['nitrogen_monoxide'].median().reset_index().rename(columns={'nitrogen_monoxide': 'nitrogen_monoxide_median'})
df = df.merge(median_monoxide_by_region_year, on=['region', 'year'], how='left')
df['nitrogen_monoxide'] = df['nitrogen_monoxide'].fillna(df['nitrogen_monoxide_median'])
df = df.drop(columns=['nitrogen_monoxide_median'])

#### 1.2 Разделение данных

In [17]:
X = df.drop(['european_aqi', 'city_id'], axis=1)
y = df['european_aqi']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

del df, X, y

#### 1.3 Обработка категориальных признаков

Преобразуем категориальные признаки с помощью LabelEncoder

In [18]:
for col in X_train.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

### 2. Обучение моделей

Обучаем модели с подбором гиперпараметров с помощью GridSearch. Также предварительно нормализуем данные с помощью StandardScaler<br>
* Linear Regression
* Elastic Net
* Decision Tree Regressor
* Random Forest REgressor
* CatBoost
* Flaml AutoML

In [105]:
results = []

In [106]:
def strip_prefix(params):
    stripped_params = {}
    for key, value in params.items():
        stripped_key = key.split('__')[-1] if '__' in key else key
        stripped_params[stripped_key] = value
    return stripped_params

#### 2.1 Linear Regression

In [108]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('Linear_Regression', LinearRegression())
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

results.append({
        "Model": 'Linear Regression',
        'MSE': mse,
        "R^2": r2,
    })

Mean Squared Error: 22.65991389373462
R^2 Score: 0.6612416002027388


#### 2.2. Elastic Net

Добавляем к модели линейной регрессии регуляризацию, обучаем Elastic Net и подбираем гиперпараметры

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('elasticnet', ElasticNet())
])

param_grid = {
    'elasticnet__alpha': [0.001, 0.01, 0.1],
    'elasticnet__l1_ratio': [0.6, 0.8, 1.0]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

results.append({
        "Model": 'Elastic Net',
        'MSE': mse,
        "R^2": r2,
        **strip_prefix(grid_search.best_params_)
    })

Best Parameters: {'elasticnet__alpha': 0.001, 'elasticnet__l1_ratio': 1.0}
Mean Squared Error: 22.660028813282782
R^2 Score: 0.6612398821925811


Регуляризация не повлияла на улучшение качества прогноза

#### 2.3 Decision Tree Regressor

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('Decision Tree Regressor', DecisionTreeRegressor(random_state=42))
])

param_grid = {
    'Decision Tree Regressor__max_depth': [30, 50, 70],
    'Decision Tree Regressor__max_leaf_nodes': [50, 70, 100],
    'Decision Tree Regressor__ccp_alpha': [0.01, 0.1]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

results.append({
        "Model": 'Decision Tree Regressor',
        'MSE': mse,
        "R^2": r2,
        **strip_prefix(grid_search.best_params_)
    })

Best Parameters: {'Decision Tree Regressor__ccp_alpha': 0.01, 'Decision Tree Regressor__max_depth': 30, 'Decision Tree Regressor__max_leaf_nodes': 100}
Mean Squared Error: 14.298053009706729
R^2 Score: 0.7862487218398522


#### 2.4 Random Forest Regressor

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('Random Forest Regressor', RandomForestRegressor(random_state=42))
])

param_grid = {
    'Random Forest Regressor__n_estimators': [20, 40],
    'Random Forest Regressor__max_depth': [30, 50],
    'Random Forest Regressor__max_leaf_nodes': [50, 70, 100],
    'Random Forest Regressor__ccp_alpha': [0.01, 0.1]
}

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=3, verbose=0, n_jobs=-1)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

results.append({
        "Model": 'Random Forest Regressor',
        'MSE': mse,
        "R^2": r2,
        **strip_prefix(grid_search.best_params_)
    })

Best Parameters: {'Random Forest Regressor__ccp_alpha': 0.01, 'Random Forest Regressor__max_depth': 30, 'Random Forest Regressor__max_leaf_nodes': 100, 'Random Forest Regressor__n_estimators': 40}
Mean Squared Error: 14.081758018637208
R^2 Score: 0.789482262152604


#### 2.5 CatBoost

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('CatBoost Regressor', CatBoostRegressor(random_state=42, verbose=0))
])

param_grid = {
    'CatBoost Regressor__iterations': [100, 200],
    'CatBoost Regressor__learning_rate': [0.01, 0.1],
    'CatBoost Regressor__depth': [6, 8],
    'CatBoost Regressor__l2_leaf_reg': [3, 5]
}

grid_search = GridSearchCV(
    estimator=pipeline, 
    param_grid=param_grid, 
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=0,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

results.append({
    "Model": 'CatBoost Regressor',
    'MSE': mse,
    "R^2": r2,
    **strip_prefix(grid_search.best_params_)
})

Best Parameters: {'CatBoost Regressor__depth': 8, 'CatBoost Regressor__iterations': 200, 'CatBoost Regressor__l2_leaf_reg': 3, 'CatBoost Regressor__learning_rate': 0.1}
Mean Squared Error: 8.76949668326902
R^2 Score: 0.8688988547183756


#### 2.6 Flaml AutoML

In [ ]:
automl = AutoML()

automl.fit(
    X_train, y_train,
    task='regression',
    time_budget=3600,
    metric='mse',
    estimator_list=['lgbm', 'xgboost', 'catboost'],
    n_jobs=-1
)

y_pred = automl.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

results.append({
    "Model": f'Flaml AutoML: {automl.model.__class__.__name__}',
    'MSE': mse,
    "R^2": r2,
    **strip_prefix(automl.best_config)
})

[flaml.automl.logger: 03-17 12:16:55] {1728} INFO - task = regression
[flaml.automl.logger: 03-17 12:16:55] {1739} INFO - Evaluation method: holdout
[flaml.automl.logger: 03-17 12:17:02] {1838} INFO - Minimizing error metric: mse
[flaml.automl.logger: 03-17 12:17:02] {1955} INFO - List of ML learners in AutoML Run: ['lgbm', 'xgboost', 'catboost']
[flaml.automl.logger: 03-17 12:17:02] {2258} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 03-17 12:17:02] {2393} INFO - Estimated sufficient time budget=1487592s. Estimated necessary time budget=2618s.
[flaml.automl.logger: 03-17 12:17:02] {2442} INFO -  at 81.9s,	estimator lgbm's best error=42.7241,	best estimator lgbm's best error=42.7241
[flaml.automl.logger: 03-17 12:17:02] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 03-17 12:17:03] {2442} INFO -  at 82.2s,	estimator lgbm's best error=42.7241,	best estimator lgbm's best error=42.7241
[flaml.automl.logger: 03-17 12:17:03] {2258} INFO - iteration 2

In [ ]:
results = pd.DataFrame(results)
results

,Model,MSE,R^2,alpha,l1_ratio,ccp_alpha,max_depth,max_leaf_nodes,n_estimators,depth,iterations,l2_leaf_reg,learning_rate,early_stopping_rounds
0,Linear Regression,22.659914,0.661242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Elastic Net,22.660029,0.661240,0.001,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Decision Tree Regressor,14.298053,0.786249,NaN,NaN,0.01,30.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Random Forest Regressor,14.081758,0.789482,NaN,NaN,0.01,30.0,100.0,40.0,NaN,NaN,NaN,NaN,NaN
4,CatBoost Regressor,8.769497,0.868899,NaN,NaN,NaN,NaN,NaN,NaN,8.0,200.0,3.0,0.1,NaN
5,Flaml AutoML: CatBoostEstimator,6.921257,0.896529,NaN,NaN,NaN,NaN,NaN,8192.0,NaN,NaN,NaN,0.1,10.0


Наилучшее качество показывает модель CatBoost<br>
* MSE = 8.77
* $R^2$ = 0.87

Также, в случае AutoML CatBoost показывает следующие результаты:<br>
* MSE = 6.92
* $R^2$ = 0.9

### 3. Снижение размерности

Протестируем различные методы снижения размерности<br>
Для проверки эффекта обучим Decision Tree и CatBoost с лучшими гиперпараметрами и сравним результаты

In [91]:
results_size = []

#### 3.1 Метод главных компонент

In [92]:
def remove_anomalies(X_train, y_train):
    pca = PCA(n_components=2)
    df_pca = pca.fit_transform(X_train)
    distances = np.sqrt(np.sum(df_pca**2, axis=1))
    threshold = np.mean(distances) + 3 * np.std(distances)
    mask = distances <= threshold
    X_train_filtered = X_train[mask]
    y_train_filtered = y_train[mask]
    return X_train_filtered, y_train_filtered

X_train_filtered, y_train_filtered = remove_anomalies(X_train, y_train)

In [93]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('DecisionTree', DecisionTreeRegressor(random_state=42, ccp_alpha=0.01, max_depth=30, max_leaf_nodes=100))
])

pipeline.fit(X_train_filtered, y_train_filtered)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

results_size.append({
        "Model": 'Decision Tree Regressor',
        'MSE PCA': mse,
        "R^2 PCA": r2
    })

Mean Squared Error: 14.594433565237694
R^2 Score: 0.7818179281839923


In [94]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('DecisionTree', CatBoostRegressor(random_state=42, verbose=0, depth=8, iterations=200, l2_leaf_reg=3, learning_rate=0.1))
])

pipeline.fit(X_train_filtered, y_train_filtered)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

results_size.append({
    "Model": 'CatBoost Regressor',
    'MSE PCA': mse,
    "R^2 PCA": r2
})

Mean Squared Error: 9.159054502123622
R^2 Score: 0.8630750910464322


#### 3.2 Isolation Forest

In [95]:
def remove_anomalies(X_train, y_train):
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    predictions = iso_forest.fit_predict(X_train)
    mask = predictions != -1
    X_train_filtered = X_train[mask]
    y_train_filtered = y_train[mask]
    return X_train_filtered, y_train_filtered

X_train_filtered, y_train_filtered = remove_anomalies(X_train, y_train)

In [96]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('DecisionTree', DecisionTreeRegressor(random_state=42, ccp_alpha=0.01, max_depth=30, max_leaf_nodes=100))
])

pipeline.fit(X_train_filtered, y_train_filtered)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

results_size.append({
        "Model": 'Decision Tree Regressor',
        'MSE Isolation Forest': mse,
        "R^2 Isolation Forest": r2
    })

Mean Squared Error: 14.911119222934635
R^2 Score: 0.777083579803709


In [97]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('DecisionTree', CatBoostRegressor(random_state=42, verbose=0, depth=8, iterations=200, l2_leaf_reg=3, learning_rate=0.1))
])

pipeline.fit(X_train_filtered, y_train_filtered)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

results_size.append({
    "Model": 'CatBoost Regressor',
    'MSE Isolation Forest': mse,
    "R^2 Isolation Forest": r2
})

Mean Squared Error: 10.628415828055488
R^2 Score: 0.8411086134229542


In [104]:
results_size = pd.DataFrame(results_size)
results_size = results_size.groupby('Model', as_index=False).first()

for index, row in results.iterrows():
    results_size.loc[results_size['Model'] == row['Model'], ['MSE', 'R^2']] = [row['MSE'], row['R^2']]

new_column_order = [
    'Model',
    'MSE', 'MSE PCA', 'MSE Isolation Forest',
    'R^2', 'R^2 PCA', 'R^2 Isolation Forest'
]

results_size = results_size[new_column_order]
results_size

,Model,MSE,MSE PCA,MSE Isolation Forest,R^2,R^2 PCA,R^2 Isolation Forest
0,CatBoost Regressor,8.769497,9.159055,10.628416,0.868899,0.863075,0.841109
1,Decision Tree Regressor,14.298053,14.594434,14.911119,0.786249,0.781818,0.777084


Видим, что MSE и $R^2$ стали хуже при снижении размерности с помощью PCA и Isolation Forest